# Assignment 2


---

This assignment is due on Feb 15th, 2020. The goal of this assignment is to use statistics to answer the important question that any technical trading strategy can yield a better return that a simple "Buy-and-Hold" strategy. Next, 


There are four technical strategies implemented in this assignement, namely, buy-and-hold, moving average crossing, Bollinger band and relative signal strength indicator. 


1.   In Buy-and-hold strategy, a trader simply buys on the first trading day and keep holding the shares. 
2.   In the moving average crossing, a trader uses 10 and 25 historical averages to create the buy and sell signals. A trader will buy if 10-days moving average is more than 25-day moving average, and sell if 10-days moving average is less than or equal to 25-day moving average
3. In Bollinger band, a trader will compute 10-day moving mean and standard deviation. If the close price is higher than the mean + standard divation, a trader will buy. Howevwer, if the close price is lower than the mean - standard deviation, a trader will sell.
4. the relative strength indicator (RSI) measures the amount of buy and sell volumes to find out whether there are the over-bought or over-sold in the market. A trader will sell a stock if the RSI is more than 70 and sell a stock if RSI is less than 40. 
The Python code for each strategies is given below.


In [1]:
import csv 
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
def buyAndHold(data_frame, is_plot=False):
  """
  Buy and Hold strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """

  close = data_frame['Adj Close']
  status = 0
  signal = []
  for k in range(close.shape[0]):
    if k == 0:
      signal.append(1)
    else:
      signal.append(0)
  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)    
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","Buy", "Sell"])
    plt.title("Buy and Hold")
    plt.grid()
  return signal


In [3]:
def movingavg(data_frame, is_plot=False):
  """
  Moving Average crossing strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  long_win = 25
  short_win = 10
  close = data_frame['Adj Close'] 
  mu_long = close.rolling(long_win).mean()
  mu_short = close.rolling(short_win).mean()
  signal = []
  status = 0
  for k in range(close.shape[0]):
    if ( mu_short[k] > mu_long[k]) and (status ==0):
      signal.append(1)
      status = 1
    elif (mu_short[k] < mu_long[k]) and (status ==1):
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)    
    plt.plot(mu_long)
    plt.plot(mu_short)
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close",f"MA({long_win})", f"MA({short_win})", "Buy", "Sell"])
    plt.title("Moving Average Crossing")
    plt.grid()
  return signal


In [4]:
def bollinger(data_frame, is_plot=False):
  """
  Bollinger Band strategy:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  close = data_frame['Adj Close']
  win = 10 
  ma = close.rolling(win).mean()
  std = close.rolling(win).std()
  up = ma + std
  low = ma - std  
  status = 0
  signal = []
  for k in range(close.shape[0]):
    closek = close[k]
    if (closek >  up[k]) and (status == 0): # buy
      signal.append(1)
      status = 1
    elif (closek < low[k]) and (status == 1): # sell
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]
  if is_plot:
    plt.figure(figsize=(15,10))
    plt.plot(close)
    plt.plot(up)
    plt.plot(low) 
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","High", "Low", "Buy", "Sell"])
    plt.title("Bollinger Bands")
    plt.grid()
  return signal


In [5]:
def rsi(data_frame, is_plot=False):
  """
  Relative Signal Strength:
  Inputs: 
          data_frame: pandas dataframe of the stock prices. It must have the "Adj Close" prices
          is_plot: If true, this code will plot the Adj close price and show the buy and sell points.
  output:
          signal indicator: the series of indicators 1 to take the position, 0 for keep previous status and -1 to clear the position.

  """
  window_length= 14
  upv = 70
  lowv= 40
  close = data_frame['Adj Close']
  delta = close.diff(1) 
 
  up, down = delta.copy(), delta.copy()
  up[up < 0] = 0
  down[down > 0] = 0
  roll_up1 = up.ewm(span=window_length).mean()
  roll_down1 = down.abs().ewm(span=window_length).mean()
  # Calculate the RSI based on EWMA
  RS1 = roll_up1 / roll_down1
  RSI1 = 100.0 - (100.0 / (1.0 + RS1))
  status = 0
  signal = []
  for k in range(close.shape[0]):
    rsk = RSI1[k]
    if (rsk < lowv) and (status == 0): # buy
      signal.append(1)
      status = 1
    elif (rsk > upv) and (status == 1): # sell
      signal.append(-1)
      status = 0
    else:
      signal.append(0)

  signal = pd.Series(data=signal, index=data_frame.index)
  buy_sig = close[signal == 1]
  sell_sig = close[signal == -1]

  if is_plot:  
    plt.figure(figsize=(15,10))
    plt.subplot(2,1,1)
    plt.plot(close)
    plt.plot(buy_sig, color="red", marker="^",linestyle="")   
    plt.plot(sell_sig, color="black", marker="v",linestyle="")  
    plt.legend(["Adj Close","Buy", "Sell"])
    plt.grid()
    plt.title("RSI")
    plt.subplot(2,1,2)
    plt.plot(RSI1)
    plt.plot(RSI1.index, upv*np.ones((close.shape[0],)),"b--")
    plt.plot(RSI1.index, lowv*np.ones((close.shape[0],)),"b--")
    plt.legend(['RSI', "SELL Band", "BUY Band"])
    plt.grid()
    plt.title("RSI Value")
  return signal


I have created a code to use the idicator to trade the stock.

In [6]:
def techicalTrading(stock_info, # dataframe of a stock
                    indicator, # trade indicator
                    initial_money=1000000 #Starting money
                    ):
  """
  Trader simulation:
  Inputs: 
        stock_info: stock price data frames
        indicator: time-series indicator: 1 for Buy, -1 for sell, 0 for nothing
        initinal_money: initial money in the port. default 1M 
  output:
        wealth: time-series money in the port
  """

  num_dates, _ = stock_info.shape
  status = "NONE" # Start with no poistion
  stock= 0
  money = initial_money
  wealth = []
  close = stock_info['Adj Close']
  for k in range(num_dates):
    indicatork = indicator[k]
    if not np.isnan(indicatork): # if it is not NA
      if (indicatork > 0) and ( status == "NONE"):
        stock = money/close.values[k]
        #print(f"Buy at {close.values[k]}")
        money = 0
        status = "HOLD"
      elif (indicatork < 0) and ( status == "HOLD"):
        money = stock * close.values[k]
        stock =  0
        #print(f"Sell at {close.values[k]}")
        status = "NONE"
    cur_wealth = money + stock * close.values[k]
    wealth.append(cur_wealth)
  wealth = pd.Series(data=wealth, index=stock_info.index)  
  return wealth 



# Question 1

Question 1: Use dataset of SET50 list in 2020 (https://www.set.or.th/th/market/files/constituents/SET50_100_H2_2020_revised.pdf) Compute the return in 2020 from 2020-01-01 to 2020-12-31. Find out

In [7]:
set50 = ['ADVANC','AOT','AWC','BBL','BDMS','BEM','BGRIM','BH','BJC','BPP',
    'BTS','CBG','CPALL','CPF','CPN','CRC','DTAC','EA','EGCO','GLOBAL',
    'GPSC','GULF','HMPRO','INTUCH','IRPC','IVL','KBANK','KTB','KTC','LH',
    'MINT','MTC','OSP','PTT','PTTEP','PTTGC','RATCH','SAWAD','SCB','SCC',
    'SCGP','TISCO','TMB','TOA','TOP','TRUE','TTW','TU','VGI','WHA']
stocks = pd.Series(set50)+'.bk'
df = data.DataReader(stocks, "yahoo", '2020-01-01', '2020-12-31')

In [8]:
BAH = buyAndHold(df, is_plot=False)
avgBAH = np.mean(BAH)
stdBAH = np.std(BAH)
print(f"Average annual return is {avgBAH*100.0}%")
print(f"Average annual STD is {stdBAH*100.0}")

Average annual return is 0.411522633744856%
Average annual STD is 6.401789788520183


In [9]:
ds = df.stack().reset_index()
MAC = movingavg(ds, is_plot=False)
avgMAC = np.mean(MAC)
stdMAC = np.std(MAC)
print(f"Average annual return is {avgMAC*100.0}%")
print(f"Average annual STD is {stdMAC*100.0}")

Average annual return is 0.0%
Average annual STD is 43.4481560052522


In [10]:
BoB = bollinger(ds, is_plot=False)
avgBoB = np.mean(BoB)
stdBoB = np.std(BoB)
print(f"Average annual return is {avgBoB*100.0}%")
print(f"Average annual STD is {stdBoB*100.0}")

Average annual return is 0.008389965601141035%
Average annual STD is 2.4234078338727754


In [11]:
plt.figure(figsize=(15,15))
RSI = rsi(ds, is_plot=False)
avgRSI = np.mean(RSI)
stdRSI = np.std(RSI)
print(f"Average annual return is {avgRSI*100.0}%")
print(f"Average annual STD is {stdRSI*100.0}")

Average annual return is 0.0%
Average annual STD is 1.2953737376634618


<Figure size 1080x1080 with 0 Axes>

# Question 2

Question 2: Compute 95% confidence interval of the average annual return for each strategies.

In [12]:
import scipy.stats as st

adj_close = df["Adj Close"]
df_ret = adj_close.pct_change(1).dropna()
num_df = df_ret.count()
df_mean = df_ret.mean()
df_std = df_ret.std()

In [13]:
numBAH = BAH.count()
sttBAH = st.t.interval(0.95, df=numBAH-1, loc=avgBAH, scale=stdBAH/np.sqrt(numBAH))
print(f"95% Confidence interval for Buy and hold is {sttBAH}.")

95% Confidence interval for Buy and hold is (-0.003974312090940128, 0.012204764765837248).


In [14]:
numMAC = MAC.count()
sttMAC = st.t.interval(0.95, df=numMAC-1, loc=avgMAC, scale=stdMAC/np.sqrt(numMAC))
print(f"95% Confidence interval for moving average crossing is {sttMAC}.")

95% Confidence interval for moving average crossing is (-0.0078008807393501155, 0.0078008807393501155).


In [15]:
numBoB = BoB.count()
sttBoB = st.t.interval(0.95, df=numBoB-1, loc=avgBoB, scale=stdBoB/np.sqrt(numBoB))
print(f"95% Confidence interval for Bollinger Band is {sttBoB}.")

95% Confidence interval for Bollinger Band is (-0.0003512100755169664, 0.0005190093875397871).


In [16]:
numRSI = RSI.count()
sttRSI = st.t.interval(0.95, df=numRSI-1, loc=avgRSI, scale=stdRSI/np.sqrt(numRSI))
print(f"95% Confidence interval for RSI is {sttRSI}.")

95% Confidence interval for RSI is (-0.00023257732823407567, 0.00023257732823407567).


# Question 3

Question 3: Use the t-test to make the conclusion whether any of these strategies yield positive return.

In [17]:
from scipy.stats import ttest_1samp
t_score, p_value = ttest_1samp(BAH, popmean=0)
print(f"Buy and Hold has t-score: {t_score:2.3f} and p-value {p_value:0.4f}. Can it make profit? no")

Buy and Hold has t-score: 1.000 and p-value 0.3183. Can it make profit? no


In [18]:
t_score, p_value = ttest_1samp(MAC, popmean=0)
print(f"Moving average crossing has t-score: {t_score:2.3f} and p-value {p_value:0.4f}. Can it make profit? no")

Moving average crossing has t-score: 0.000 and p-value 1.0000. Can it make profit? no


In [19]:
t_score, p_value = ttest_1samp(BoB, popmean=0)
print(f"Bollinger Band has t-score: {t_score:2.3f} and p-value {p_value:0.4f}. Can it make profit? no")

Bollinger Band has t-score: 0.378 and p-value 0.7055. Can it make profit? no


In [20]:
t_score, p_value = ttest_1samp(RSI, popmean=0)
print(f"RSI has t-score: {t_score:2.3f} and p-value {p_value:0.4f}. Can it make profit? no")

RSI has t-score: 0.000 and p-value 1.0000. Can it make profit? no


# Question 4

Question 4: Next, we compare againts Buy-and-Hold startegy using t-score. Here,

H0 : A strategy is the same or worst than buy-and-hold

H1: A strategy is better than buy and hold:


In [21]:
from scipy.stats import t
t_score = t.ppf(1-0.025, numMAC+numBAH-2)
SP2 = ((numMAC-1)*stdMAC**2+(numBAH-1)*stdBAH**2)/numMAC+numBAH-2
scales = np.sqrt(SP2*(1/numMAC+1/numBAH))
p_val = (1-t.cdf(x=avgMAC-avgBAH, loc=0, scale=scales, df=numMAC+numBAH-2))*2.0
print(f"Moving Average vs Buy and Hold: t-score {t_score} p-value {p_val}. Is moving average better than buy and hold? no")

Moving Average vs Buy and Hold: t-score 1.960159091724343 p-value 1.0032626132081521. Is moving average better than buy and hold? no


In [22]:
t_score = t.ppf(1-0.025, numBoB+numBAH-2)
SP2 = ((numBoB-1)*stdBoB**2+(numBAH-1)*stdBAH**2)/numBoB+numBAH-2
scales = np.sqrt(SP2*(1/numBoB+1/numBAH))
p_val = t.cdf(x=avgBoB-avgBAH, loc=0, scale=scales, df=numBoB+numBAH-2)*2.0
print(f"Bolling Band vs Buy and Hold: t-score {t_score} p-value {p_val}. Is bolling band better than buy and hold? no")

Bolling Band vs Buy and Hold: t-score 1.960159091724343 p-value 0.9968026558473501. Is bolling band better than buy and hold? no


In [23]:
t_score = t.ppf(1-0.025, numRSI+numBAH-2)
SP2 = ((numRSI-1)*stdRSI**2+(numBAH-1)*stdBAH**2)/numRSI+numBAH-2
scales = np.sqrt(SP2*(1/numRSI+1/numBAH))
p_val = t.cdf(x=avgRSI-avgBAH, loc=0, scale=scales, df=numRSI+numBAH-2)*2.0
print(f"RSI vs Buy and Hold: t-score {t_score} p-value {p_val}. Is RSI better than buy and hold? no")

RSI vs Buy and Hold: t-score 1.960159091724343 p-value 0.99673611049767. Is RSI better than buy and hold? no
